# Exploring the SVHN Data

### Importing neccessary libraries

In [11]:

import numpy as np
import scipy.io as sio


### Loading the test and training data

In [20]:
train = sio.loadmat("data/train_32x32.mat")
Xtr = train.get('X')
Ytr = train.get('y')

test = sio.loadmat("data/test_32x32.mat")
Xva = test.get('X')
Yva = test.get('y')

In [22]:
print("Training data:")
print(Xtr.shape)
print(Ytr.shape)
print("Validation data:")
print(Xva.shape)
print(Yva.shape)

Training data:
(32, 32, 3, 73257)
(73257, 1)
Validation data:
(32, 32, 3, 26032)
(26032, 1)
